### Imports

In [14]:
import pandas as pd
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np


### Setup Data

In [15]:
root = "/home/GTL/jmagana/gte/ml/TurtlebotFollower/data"

In [16]:
class GestureDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data = []
        self.labels = []
        self.transform = transform

        data_dir = Path(data_dir)
        self.class_map = {folder.name: idx for idx, folder in enumerate(sorted(data_dir.iterdir())) if folder.is_dir()}

        for class_name, label in self.class_map.items():
            csv_file = data_dir / class_name / "data.csv"
            df = pd.read_csv(csv_file)
            for _, row in df.iterrows():
                self.data.append(row.values.astype(np.float32))
                self.labels.append(label)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.labels[idx]
        if self.transform:
            x = self.transform(x)
        return torch.tensor(x), torch.tensor(y)

In [17]:
dataset = GestureDataset(root)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [18]:
print(dataset.class_map)

{'follow': 0, 'other': 1, 'stop': 2}


### Train

In [19]:
class GestureModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.model = nn.Sequential(
            nn.LayerNorm(input_size),
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes),
        )
    
    def forward(self, x):
        return self.model(x)

In [20]:
model = GestureModel(165, len(dataset.class_map))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [21]:
EPOCHS = 100

model.train()
for epoch in range(EPOCHS):
    total_loss = 0.
    for X, y in dataloader:
        logits = model(X)
        loss = criterion(logits, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch: {epoch + 1} Loss: {total_loss / len(dataloader)}")

Epoch: 1 Loss: 1.0733340978622437
Epoch: 2 Loss: 0.953346868356069
Epoch: 3 Loss: 0.923482080300649
Epoch: 4 Loss: 0.9169543186823527
Epoch: 5 Loss: 0.8990113337834676
Epoch: 6 Loss: 0.8676184813181559
Epoch: 7 Loss: 0.8546401063601176
Epoch: 8 Loss: 0.8258344729741415
Epoch: 9 Loss: 0.8114627997080485
Epoch: 10 Loss: 0.794398566087087
Epoch: 11 Loss: 0.7760029037793478
Epoch: 12 Loss: 0.7531235416730245
Epoch: 13 Loss: 0.7383865118026733
Epoch: 14 Loss: 0.7176188826560974
Epoch: 15 Loss: 0.6968949238459269
Epoch: 16 Loss: 0.6803903182347616
Epoch: 17 Loss: 0.6609137455622355
Epoch: 18 Loss: 0.6382696628570557
Epoch: 19 Loss: 0.622866690158844
Epoch: 20 Loss: 0.5932791928450266
Epoch: 21 Loss: 0.5703293482462565
Epoch: 22 Loss: 0.5479931632677714
Epoch: 23 Loss: 0.5339322288831075
Epoch: 24 Loss: 0.5010971029599508
Epoch: 25 Loss: 0.4719899197419484
Epoch: 26 Loss: 0.45643848180770874
Epoch: 27 Loss: 0.4279033641020457
Epoch: 28 Loss: 0.405046949783961
Epoch: 29 Loss: 0.400747110446294

### Save model

In [22]:
model.eval()
dummy_input = torch.randn(1, 165)
torch.onnx.export(
    model,
    dummy_input,
    "gesture_mlp.onnx",
    input_names=['input'],
    output_names=['output'],
    opset_version=17
)

W1113 09:24:49.237000 3411882 torch/onnx/_internal/exporter/_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 17 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features


[torch.onnx] Obtain model graph for `GestureModel([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `GestureModel([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


The model version conversion is not supported by the onnxscript version converter and fallback is enabled. The model will be converted using the onnx C API (target version: 17).


[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅


ONNXProgram(
    model=
        <
            ir_version=10,
            opset_imports={'': 17},
            producer_name='pytorch',
            producer_version='2.9.0+cu128',
            domain=None,
            model_version=None,
        >
        graph(
            name=main_graph,
            inputs=(
                %"input"<FLOAT,[1,165]>
            ),
            outputs=(
                %"output"<FLOAT,[1,3]>
            ),
            initializers=(
                %"model.0.weight"<FLOAT,[165]>{TorchTensor(...)},
                %"model.0.bias"<FLOAT,[165]>{TorchTensor(...)},
                %"model.1.bias"<FLOAT,[128]>{TorchTensor(...)},
                %"model.3.bias"<FLOAT,[64]>{TorchTensor(...)},
                %"model.5.weight"<FLOAT,[3,64]>{TorchTensor(...)},
                %"model.5.bias"<FLOAT,[3]>{TorchTensor<FLOAT,[3]>(Parameter containing: tensor([-0.0744, -0.0085, -0.0727], requires_grad=True), name='model.5.bias')},
                %"model.1.weight"<FLOAT,